In [ ]:
# !pip install mxnet-cu101
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece==0.1.85
# !pip install transformers==2.1.1
# !pip install torch==1.3.1

In [1]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

     |████████████████████████████████| 710.5 MB 8.5 kB/s eta 0:00:01
  Using cached gluonnlp-0.9.1.tar.gz (252 kB)
  Using cached Cython-0.29.20-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp38-cp38-linux_x86_64.whl size=501287 sha256=671568010f49623a116f770d45c681015de6a0a5b65e44560a8a8fa26e2a4827
  Stored in directory: /home/boychaboy/.cache/pip/wheels/92/fb/71/9b3afdece6b125729ae59b1524d9adcf932d7380e688b449b7
Successfully built gluonnlp


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-m31hcvl3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-m31hcvl3
  Created wheel for kobert: filename=kobert-0.1.1-py3-none-any.whl size=12871 sha256=ad21d15c22a91bb880534a1fd4d7afcbf811c112ffebf8d98708110086662e21
  Stored in directory: /tmp/pip-ephem-wheel-cache-hkd99g66/wheels/bf/5f/74/81bf3a1332130eb6629ecf58876a8746b77021e7d7b0638e91
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
# from transformers.optimization import WarmupLinearSchedule
from transformers import get_linear_schedule_with_warmup

In [7]:
##GPU 사용 시
device = torch.device("cuda:0")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [9]:
dataset_train = nlp.data.TSVDataset("nsmc/ratings_train.txt", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("nsmc/ratings_test.txt", field_indices=[1,2], num_discard_samples=1)

In [14]:
dataset_train[100]

['신카이 마코토의 작화와,미유와 하나카나가 연기를 잘해줘서 더대박이였다.', '1']

In [15]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [17]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [19]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [22]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [24]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [26]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [27]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [28]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-28-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


epoch 1 batch id 1 loss 0.7093364000320435 train acc 0.578125
epoch 1 batch id 201 loss 0.5101841688156128 train acc 0.5788246268656716
epoch 1 batch id 401 loss 0.488308846950531 train acc 0.6778366583541147
epoch 1 batch id 601 loss 0.4188087284564972 train acc 0.7266014975041597
epoch 1 batch id 801 loss 0.43801769614219666 train acc 0.7558325530586767
epoch 1 batch id 1001 loss 0.292766273021698 train acc 0.7750062437562437
epoch 1 batch id 1201 loss 0.36145180463790894 train acc 0.7884315154038302
epoch 1 batch id 1401 loss 0.3802943229675293 train acc 0.7975999286224126
epoch 1 batch id 1601 loss 0.351998895406723 train acc 0.8059123204247346
epoch 1 batch id 1801 loss 0.23608028888702393 train acc 0.8128730566352027
epoch 1 batch id 2001 loss 0.32831335067749023 train acc 0.8189186656671664
epoch 1 batch id 2201 loss 0.27901989221572876 train acc 0.824156633348478

epoch 1 train acc 0.8276806029579067


<ipython-input-28-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):



epoch 1 test acc 0.8811141304347826


epoch 2 batch id 1 loss 0.4304448664188385 train acc 0.8125
epoch 2 batch id 201 loss 0.19990697503089905 train acc 0.8800528606965174
epoch 2 batch id 401 loss 0.263326495885849 train acc 0.8834164588528678
epoch 2 batch id 601 loss 0.38143739104270935 train acc 0.88573731281198
epoch 2 batch id 801 loss 0.3397142291069031 train acc 0.8880305867665418
epoch 2 batch id 1001 loss 0.23441912233829498 train acc 0.8909059690309691
epoch 2 batch id 1201 loss 0.19954019784927368 train acc 0.8938514779350542
epoch 2 batch id 1401 loss 0.23695439100265503 train acc 0.8968705389007852
epoch 2 batch id 1601 loss 0.35101765394210815 train acc 0.8988230012492192
epoch 2 batch id 1801 loss 0.18626707792282104 train acc 0.9007756107717935
epoch 2 batch id 2001 loss 0.21856847405433655 train acc 0.9027127061469266
epoch 2 batch id 2201 loss 0.25592467188835144 train acc 0.9041273852794185

epoch 2 train acc 0.9053856477531286


...


epoch 3 test acc 0.895400415601023


epoch 4 batch id 1 loss 0.41443440318107605 train acc 0.859375
epoch 4 batch id 201 loss 0.06927099078893661 train acc 0.9561567164179104
epoch 4 batch id 401 loss 0.06823982298374176 train acc 0.9555798004987531
epoch 4 batch id 601 loss 0.23131349682807922 train acc 0.9561408069883528
epoch 4 batch id 801 loss 0.13685646653175354 train acc 0.9575335518102372
epoch 4 batch id 1001 loss 0.06695158779621124 train acc 0.9587912087912088
epoch 4 batch id 1201 loss 0.05230768769979477 train acc 0.9597991257285595
epoch 4 batch id 1401 loss 0.08275384455919266 train acc 0.9608650071377588
epoch 4 batch id 1601 loss 0.10209817439317703 train acc 0.9616255465334166
epoch 4 batch id 1801 loss 0.08631472289562225 train acc 0.9623993614658523
epoch 4 batch id 2001 loss 0.08201278001070023 train acc 0.9632058970514743
epoch 4 batch id 2201 loss 0.14189597964286804 train acc 0.9636457860063607

epoch 4 train acc 0.96417715443686



epoch 4 test acc 0.8964194373401535


epoch 5 batch id 1 loss 0.2862774431705475 train acc 0.921875
epoch 5 batch id 201 loss 0.02337711676955223 train acc 0.9735696517412935
epoch 5 batch id 401 loss 0.04109572619199753 train acc 0.973698566084788
epoch 5 batch id 601 loss 0.27604761719703674 train acc 0.9742096505823628
epoch 5 batch id 801 loss 0.0688333585858345 train acc 0.9746215667915106
epoch 5 batch id 1001 loss 0.009490116499364376 train acc 0.9753996003996004
epoch 5 batch id 1201 loss 0.010796567425131798 train acc 0.9760095753538718
epoch 5 batch id 1401 loss 0.054756347090005875 train acc 0.9762000356887938
epoch 5 batch id 1601 loss 0.03840545564889908 train acc 0.9767235321673954
epoch 5 batch id 1801 loss 0.10762347280979156 train acc 0.9772001665741255
epoch 5 batch id 2001 loss 0.04991726577281952 train acc 0.9773160294852574
epoch 5 batch id 2201 loss 0.10824605077505112 train acc 0.977538618809632

epoch 5 train acc 0.9777690379692833



epoch 5 test acc 0.8963395140664961
